In [29]:
import os
import tarfile


from fnmatch import fnmatch
from pathlib import Path

#Root folder with all the tarball files (h5 Files)
root = '/Volumes/Samsung_T5/1M_SONGS'
pattern = "A.tar.gz"
pattern_h5 = "*.h5"

print('Working directory: ' + root)
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            f_data= os.path.join(path, name)
            print('Working tar file: ' + f_data)
            t_file = tarfile.open(f_data, "r:gz")
            t_file.extractall(root)
            t_file.close()
            ind_dot = name.index('.')
            if( name[:ind_dot]):
                n_dir = name[:ind_dot]
                f_h5= os.path.join(path, n_dir)
                print('Searching for files in directory: ' + f_h5) 
                for path_h5, subdirs_h5, files_h5 in os.walk(root):
                    for n in files_h5:
                        if fnmatch(n, pattern_h5):
                            f_h5= os.path.join(path_h5, n)
                            print('Encountered H5 file: ' + f_h5)
                



Working directory: /Volumes/Samsung_T5/1M_SONGS
Working tar file: /Volumes/Samsung_T5/1M_SONGS/A.tar.gz
Searching for files in directory: /Volumes/Samsung_T5/1M_SONGS/A
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEGO12903CF7D27.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEIJ128F92DFEDB.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEMK12903CF5AD5.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEVW128F4244269.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEDH128F932BF7D.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEPV12903D0D41E.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEUO128F92EF8AD.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEKM12903CA3944.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEWM12903CBB182.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEELW128F933D255.h5
Encountered H5 file: /Volumes/Samsung_T5/1M_SONGS/A

In [4]:
import h5py
import numpy as np
import mysql.connector
from mysql.connector import errorcode

config = {
  'user': 'ivan',
  'password': '********',
  'host': 'musicmood-instance.ctjjankvidir.us-east-1.rds.amazonaws.com',
  'database': 'musicmood',
  'raise_on_warnings': True,
}

filename = '/Volumes/Samsung_T5/1M_SONGS/A/E/E/TRAEEGO12903CF7D27.h5'
f = h5py.File(filename, 'r')

data_set = f['analysis']
arrlev = data_set['songs']

track_id = arrlev['track_id']
danceability = arrlev['danceability']
duration = arrlev['duration']
end_of_fade_in = arrlev['end_of_fade_in']
energy = arrlev['energy']
key = arrlev['key']
key_confidence = arrlev['key_confidence']
loudness = arrlev['loudness']
mode = arrlev['mode']
mode_confidence = arrlev['mode_confidence']
start_of_fade_out = arrlev['start_of_fade_out']
tempo = arrlev['tempo']
time_signature = arrlev['time_signature']
time_signature_confidence = arrlev['time_signature_confidence']

data_song ={
    'track_id':track_id,
    'danceability':danceability,
    'duration':duration,
    'end_of_fade_in':end_of_fade_in,
    'energy':energy,
    'key':key,
    'key_confidence':key_confidence,
    'loudness':loudness,
    'mode':mode,
    'mode_confidence':mode_confidence,
    'start_of_fade_out':start_of_fade_out,
    'tempo':tempo,
    'time_signature':time_signature,
    'time_signature_confidence':time_signature_confidence
}

try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    sql = "INSERT INTO `musicmood`.`songs_new_features`(`track_id`,`danceability`,`duration`,`end_of_fade_in`,`energy`,`key`,`key_confidence`,`loudness`,`mode`,`mode_confidence`,`start_of_fade_out`,`tempo`,`time_signature`,`time_signature_confidence`) VALUES(%s,%f,%f,%f,%f,%i,%f,%f,%i,%f,%f,%f,%i,%f)"
    cursor.execute(sql, data_song)
    cnx.commit()
    cursor.close()
    cnx.close()
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cnx.close()


ModuleNotFoundError: No module named 'mysql'